In [472]:
## This script takes the command-line output of a blast xml search and merges it with the two tissues data
## The issue is that the blast search doesn't always give the HUGO gene symbol, even where one is available

# Import libraries
import pandas as pd
import numpy as np
import StringIO as st
import fnmatch
import re
from Bio.Blast import NCBIXML

# Read in files
f = open( 'V:/Daher_Grace/For ME/two_tissue_annotation/blast_from_fasta.xml', 'r')
raws = NCBIXML.parse(f)


rlist = []
n = 13
for raw in raws:
    x = re.split("[;/:]",raw.query)
    gene = fnmatch.filter(x,"GEN=*")
    #definition = fnmatch.filter(x,"DEF=*")
    if gene:
        gene = re.split("=",gene[0])[1]
    else:
        gene = ""
    for alignment in raw.alignments:
        for hsp in alignment.hsps:
            enum = hsp.expect
            y=hsp.query
          
    row = [x[2],gene,enum,y,x]
    rlist.append(row)
    #if(len(x)!= n):
            #raise Exception("Rows not equal")
f.close()   

f = open("//jdcfs1/cores/Bioinformatics/patti/two_tissue/ncRNA_cor_gene_stats.csv")
dat = pd.read_csv(f,index_col=0, dtype = object)
f.close()

annot = pd.DataFrame(rlist,columns=["Probeset","Symbol","E value","Sequence","Query"])
annot.index.names = [None]
annot.set_index('Probeset', inplace=True)
print(sum(pd.isnull(dat.Symbol)))
print(sum(pd.notnull(dat.Symbol)))
print(annot)



   
            


12315
42297
               Symbol        E value  \
Probeset                               
1552256_a_at  SCARB1    0.000000e+00   
1552257_a_at            0.000000e+00   
1552258_at             3.051930e-111   
1552261_at     WFDC2    2.736600e-75   
1552263_at     MAPK1    7.819550e-60   
1552264_a_at   MAPK1    0.000000e+00   
1552266_at              1.365550e-62   
1552269_at             3.081190e-152   
1552271_at              5.858460e-98   
1552272_a_at           3.403660e-172   
1552274_at              0.000000e+00   
1552275_s_at           1.340160e-104   
1552276_a_at   VPS18   1.567730e-170   
1552277_a_at            0.000000e+00   
1552278_a_at            0.000000e+00   
1552279_a_at            0.000000e+00   
1552280_at              0.000000e+00   
1552281_at              0.000000e+00   
1552283_s_at            1.043610e-04   
1552286_at              0.000000e+00   
1552287_s_at  AFG3L1    0.000000e+00   
1552288_at              6.555990e-50   
1552289_a_at           5.915

In [473]:
# Add sequence to data

merged = pd.merge(left=dat,right=annot,how='left',left_index=True,right_index=True)
print(merged)
merged.to_csv("V:/Daher_Grace/For ME/two_tissue_annotation/ncRNA_cor_gene_stats_sequences.csv")



                   cor       p FDR      Symbol_x     Entrez  \
1007_s_at       0.0386   0.813   1           NaN        NaN   
1053_at        0.00745   0.964   1          RFC2       5982   
117_at           0.165   0.308   1         HSPA6       3310   
121_at         -0.0185    0.91   1          PAX8       7849   
1255_g_at       0.0318   0.845   1        GUCA1A       2978   
1294_at         0.0432   0.791   1           NaN        NaN   
1316_at        -0.0118   0.942   1          THRA       7067   
1320_at         -0.132   0.417   1        PTPN21      11099   
1405_i_at      -0.0851   0.602   1          CCL5       6352   
1431_at         0.0231   0.888   1        CYP2E1       1571   
1438_at        -0.0644   0.693   1         EPHB3       2049   
1487_at          0.148   0.362   1         ESRRA       2101   
1494_f_at       0.0142   0.931   1        CYP2A6       1548   
1552256_a_at     0.104   0.524   1        SCARB1        949   
1552257_a_at   -0.0906   0.578   1        TTLL12      2

In [426]:
#update data with new annotations

changed = 0
total = 0
cols = ['Symbol']
for probe in dat.index:
    total = total+1
    if (pd.isnull(dat.loc[probe]['Symbol'])& pd.notnull(annot.loc[probe]['Symbol'])):
        dat.loc[probe,cols] == annot.loc[probe,cols]
        changed=changed+1

print(total)
print(changed)

print(dat)



54612
12315
                    cor         p FDR        Symbol     Entrez  \
242749_at        -0.595  5.16e-05   1           NaN        NaN   
241295_at         0.565  0.000146   1           NaN        NaN   
238407_at        -0.561  0.000168   1           NaN        NaN   
1563320_at       -0.559  0.000179   1           NaN        NaN   
241301_at        -0.545  0.000279   1           NaN        NaN   
216392_s_at       0.543  0.000297   1       SEC23IP      11196   
217297_s_at      -0.542  0.000301   1         MYO9B       4650   
1556227_at       -0.541  0.000317   1        VCPIP1      80124   
233935_at        -0.534  0.000387   1       TNFSF14       8740   
233242_at        -0.526  0.000488   1       SCAND2P      54581   
215029_at        -0.522  0.000549   1           NaN        NaN   
217237_at         0.521  0.000561   1           NaN        NaN   
1561978_at        0.521  0.000562   1     LOC284798     284798   
232409_x_at       0.519  0.000603   1        FBXL16     146330  

In [460]:
# Here too there do not seem to be any new annotation.
print(sum(pd.isnull(dat.Symbol)))
print(sum(pd.notnull(dat.Symbol)))
print(sum(pd.isnull(annot.Symbol)))
print(sum(pd.notnull(annot.Symbol)))
print(len(annot))
print(len(dat))

12315
42297
0
54675
54675
54612


In [432]:
print(annot.columns.values)
print(dat.columns.values)

['Symbol' 'E value' 'Query']
['cor' 'p' 'FDR' 'Symbol' 'Entrez' 'Description']


In [462]:
print(annot.loc['242684_at']['Symbol'])

In [379]:
## I am testing whether it would be better to just update the annotation using ortholog and annotation file from affy
## I found that the annotation was already updated as much as possible, so there was no point to this.
#read from data file
f = open("//jdcfs1/cores/Bioinformatics/patti/two_tissue/ncRNA_cor_gene_stats.csv")
data = pd.read_csv(f,index_col=0, dtype = object)
f.close()


print(sum(pd.notnull(data.iloc[:]['Symbol'])))
print(sum(pd.isnull(data.iloc[:]['Symbol'])))


42297
12315


In [380]:
# read from annotation file
f= open("V:/Daher_Grace/For ME/two_tissue_annotation/HG-U133_Plus_2-na36-annot-csv/HG-U133_Plus_2.na36.annot.csv")
annotation = pd.read_csv(f,skiprows = 25,usecols=['Probe Set ID','Gene Symbol','Entrez Gene','Gene Title'], index_col='Probe Set ID',dtype =object)
annotation.columns = ['Description','Symbol','Entrez']

f.close()

In [381]:
import math


#update data with new annotations
changed = 0
total = 0
for probe in data.index:
    total = total+1
    if (pd.isnull(data.loc[probe]['Symbol'])& pd.notnull(annotation.loc[probe]['Symbol'])):
        cols = ['Symbol','Description','Entrez']
        #probe = data.index[row]
        data.loc[probe,cols] == annotation.loc[probe,cols]
        changed=changed+1
print(total)
print(changed)

print(sum(pd.notnull(annotation.iloc[:]['Symbol'])))
print(sum(pd.isnull(annotation.iloc[:]['Symbol'])))
print(sum(pd.notnull(data.iloc[:]['Symbol'])))
print(sum(pd.isnull(data.iloc[:]['Symbol'])))


data = data.fillna('')
print(data)



54612
12315
54675
0
42297
12315
                    cor         p FDR        Symbol     Entrez  \
242749_at        -0.595  5.16e-05   1                            
241295_at         0.565  0.000146   1                            
238407_at        -0.561  0.000168   1                            
1563320_at       -0.559  0.000179   1                            
241301_at        -0.545  0.000279   1                            
216392_s_at       0.543  0.000297   1       SEC23IP      11196   
217297_s_at      -0.542  0.000301   1         MYO9B       4650   
1556227_at       -0.541  0.000317   1        VCPIP1      80124   
233935_at        -0.534  0.000387   1       TNFSF14       8740   
233242_at        -0.526  0.000488   1       SCAND2P      54581   
215029_at        -0.522  0.000549   1                            
217237_at         0.521  0.000561   1                            
1561978_at        0.521  0.000562   1     LOC284798     284798   
232409_x_at       0.519  0.000603   1       

In [373]:
data.to_csv("V:/Daher_Grace/For ME/two_tissue_annotation/ncRNA_cor_gene_stats_annot_update.csv")

In [371]:
print(sum(pd.notnull(annotation.iloc[:]['Symbol'])))
print(sum(pd.isnull(annotation.iloc[:]['Symbol'])))
print(sum(pd.notnull(data.iloc[:]['Symbol'])))
print(sum(pd.isnull(data.iloc[:]['Symbol'])))

54675
0
54612
0


In [307]:
print(data.index.values)
print(annotation.index.values)
print(data.columns)
print(annotation.columns.values)

['242749_at' '241295_at' '238407_at' ..., '207275_s_at' '215833_s_at'
 '219452_at']
['1007_s_at' '1053_at' '117_at' ..., 'AFFX-TrpnX-3_at' 'AFFX-TrpnX-5_at'
 'AFFX-TrpnX-M_at']
Index([u'cor', u'p', u'FDR', u'Symbol', u'Entrez', u'Description'], dtype='object')
['Symbol' 'Entrez' 'Description']
